In [2]:
from finlab import data

#guide can be found here
#https://mrjbq7.github.io/ta-lib/funcs.html
#https://twopirllc.github.io/pandas-ta/#indicators-by-category


In [6]:
from finlab import data
import finlab
finlab.login(api_token="QNBAVxVcsFjg0vZ5Um8P4XVtRWnhnMtznBpGjGHXqQ+vBYWkN6kXnXDrW70KhH4v#free")

輸入成功!


In [11]:
#MOMENT=data.indicator("MOM",timeperiod=14)
APX=data.indicator("adx",timeperiod=14)

In [37]:
from finlab import data

close = data.get('price:收盤價')
position = close >= close.rolling(300).max()
position

,0015,0050,0051,0052,0053,0054,0055,0056,0057,0058,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2007-04-23,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2007-04-24,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2007-04-25,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2007-04-26,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2007-04-27,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2022-07-11,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2022-07-12,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [36]:
from finlab import backtest
